In [1]:
from pathlib import Path
import pandas as pd
import re

In [4]:
notebook_path = Path().resolve().parent.parent.parent
print("Current Notebook Path:", notebook_path)
data_path = notebook_path / "data" / "March_1"
print("Data Path:", data_path)

Current Notebook Path: C:\Users\Vignesh\Desktop\SDM
Data Path: C:\Users\Vignesh\Desktop\SDM\data\March_1


In [6]:
class_options = ["Class A", "Class B", "Class C", "Other"]
print("Select the class name:")
for i, option in enumerate(class_options, 1):
    print(f"{i}. {option}")
class_choice = int(input("Enter the number corresponding to your choice: ") or 1)
class_name = class_options[class_choice - 1] if 1 <= class_choice <= len(class_options) else "Other"

# Replace 'your_file.csv' with the actual path to your CSV file
file_path = f'{data_path}/march_daniel_issac_gsn.csv'

location = input("Enter the location of the header row (0-indexed): ")
ignore_rows = int(input("Enter the number of rows to ignore from the end: "))

location = int(location)
# Read the CSV file
df = pd.read_csv(file_path,header =location)
if ignore_rows > 0:
    df = df.iloc[:-ignore_rows]

# Display all columns in the dataframe
print("Columns in the CSV file:")
print(df.columns.tolist())

# Display the first few rows of the dataframe
df.head()


Select the class name:
1. Class A
2. Class B
3. Class C
4. Other
Columns in the CSV file:
['Student Name', 'Mobile', 'Staff', 'Networker', 'DOB', 'Age', 'Gender', 'Position', 'Aadhaar Name', 'Aadhaar DOB', 'BB -Cummulative Value', '18 March 2025', '13 March 2025', '10 March 2025', '6 March 2025', '3 March 2025', '27 February 2025', '24 February 2025', '20 February 2025']


,Student Name,Mobile,Staff,Networker,DOB,Age,Gender,Position,Aadhaar Name,Aadhaar DOB,BB -Cummulative Value,18 March 2025,13 March 2025,10 March 2025,6 March 2025,3 March 2025,27 February 2025,24 February 2025,20 February 2025
0,Arun,44440570034,NaN,A.Nevis Jeyaseelan,NaN,NaN,NaN,NaN,NaN,NaN,8,O,O,O,O,O,O,O,O
1,Swapna Gopinath,8111024659,NaN,Aarti Vignesh,20-May-1998,26.0,Female,Believer,NaN,NaN,8,O,O,O,O,O,O,O,O
2,Gipsy,9952527607,NaN,Aarti Vignesh,27-Jan-2001,24.0,Female,Believer,NaN,NaN,4,X,X,O,O,X,O,O,X
3,Iniya .S.,6381641661,NaN,Abin C Paul,NaN,NaN,NaN,NaN,NaN,NaN,6,X,O,O,O,O,O,O,X
4,Mugesh,9025159504,NaN,Abin C Paul,15-Jun-2002,22.0,Male,Seminar Student,NaN,NaN,5,X,O,X,O,O,O,O,X


In [7]:
patterns = {
    "student_name": re.compile(r'name|student', re.IGNORECASE),
    "mobile_phone": re.compile(r'mobile|phone|number', re.IGNORECASE),
    "staff": re.compile(r'staff', re.IGNORECASE),
    "networker": re.compile(r'networker', re.IGNORECASE)
}

In [8]:
matched_columns = {}
for key, pattern in patterns.items():
    for column in df.columns:
        if pattern.search(column):
            matched_columns[key] = column
            break

In [9]:
print("Matched Columns:")
print(matched_columns)

Matched Columns:
{'student_name': 'Student Name', 'mobile_phone': 'Mobile', 'staff': 'Staff', 'networker': 'Networker'}


In [10]:
# Keep only matched columns
df = df[[col for col in matched_columns.values() if col in df.columns]]

# Normalize mobile numbers and determine country if a matching column exists
if "mobile_phone" in matched_columns:
    mobile_col = matched_columns["mobile_phone"]
    df[mobile_col] = df[mobile_col].astype(str).str.replace(r'\D', '', regex=True)  # Remove non-numeric characters
    
    # Determine country
    def get_country(phone):
        if len(phone) == 10:
            return "India"
        elif len(phone) == 9 and phone.startswith("7"):
            return "Sri Lanka"
        elif phone.startswith("94") and len(phone) == 11:
            return "Sri Lanka"
        else:
            return "Others"
    
    df["Country"] = df[mobile_col].apply(get_country)

In [11]:
if "staff" in df.columns:
    df["staff"] = df["staff"].astype(str).str.replace(r'^\d+\.\s*', '', regex=True).str.strip()
display(df.head())

,Student Name,Mobile,Staff,Networker,Country
0,Arun,44440570034,NaN,A.Nevis Jeyaseelan,Others
1,Swapna Gopinath,8111024659,NaN,Aarti Vignesh,India
2,Gipsy,9952527607,NaN,Aarti Vignesh,India
3,Iniya .S.,6381641661,NaN,Abin C Paul,India
4,Mugesh,9025159504,NaN,Abin C Paul,India


In [14]:
df["Class"] = class_name
df['STAFF ASSIGNED'] = df['STAFF ASSIGNED'].str.replace(r'^\d+\.', '', regex=True)

df.head(10)

,Name,number,STAFF ASSIGNED,Networker\n(인도자),Country,Class
0,Sivaranjan,7604873595,NaN,NaN,India,Class A
1,Santhosh,6381965464,NaN,NaN,India,Class A
2,Jacob abraham,7871852560,Johnson Isaac,NaN,India,Class A
3,Aravinth Manoharan,7868940673,Johnson Isaac,NaN,India,Class A
4,Karthick,7358210334,NaN,NaN,India,Class A
5,Ajith Kumar,7299793729,NaN,NaN,India,Class A
6,Ebenesar,7708048581,Johnson Isaac,NaN,India,Class A
7,antony,7200200382,NaN,NaN,India,Class A
8,Nirmal R,7094748846,NaN,NaN,India,Class A
9,Sivaranjan,7604873595,Johnson Isaac,NaN,India,Class A


In [15]:
df.count()

Name                505
number              505
STAFF ASSIGNED      311
Networker\n(인도자)    108
Country             505
Class               505
dtype: int64